In [0]:
%sql
--CREATE OR REPLACE TABLE workspace.bigquery_db_cohort_db.gold_retention_rate AS

WITH 
customer_groups AS (
  SELECT 
      customer_id,
      first_order_date,
      second_order_date,
      -- monthly cohort
      DATE_TRUNC('month', first_order_date)::DATE AS cohort_month, 
      -- difference in months between first and second purchase (anstelle von DATE_DIFF)
      FLOOR(months_between(second_order_date, first_order_date)) AS months_difference

  FROM workspace.bigquery_db_cohort_db.silver_cohort_analysis

)


SELECT 
    cohort_month,
    --number of customers in each cohort
    COUNT(*) AS num_customer,
    
    --percentages of customers who made a second purchase within 1,2 or 3 month
    ROUND(100.0 * SUM(CASE WHEN months_difference  <= 1 THEN 1 END) / COUNT(*),2) AS retention_rate_1m,
    ROUND(100.0 * SUM(CASE WHEN months_difference  <= 2 THEN 1 END) / COUNT(*),2) AS retention_rate_2m,
    ROUND(100.0 * SUM(CASE WHEN months_difference  <= 3 THEN 1 END) / COUNT(*),2) AS retention_rate_3m

FROM customer_groups
GROUP BY cohort_month
ORDER BY cohort_month;